Library Imports

In [1]:
import csv
import pandas as pd

In [8]:
#code for reading file
def readf(file):
    f = open(file, encoding="utf-8")
    data = list(csv.reader(f))
    f.close()
    return data



In [9]:

scores = {}
count = {}
conversion = {"Not a deficiency": 0, "Low": 1, "Medium": 2, "High": 3}
data = readf("psc_severity_train.csv")
print(data[5015][4])
for row in data[1:]:
    if row[4] != '':
        if row[0] not in scores:
            count[row[0]] = 1
            scores[row[0]] = conversion[row[4]]
            
        else:
            count[row[0]] += 1
            scores[row[0]] += conversion[row[4]]
print(scores)
print(count)

Low
{'1702496': 18, '1795901': 15, '1667488': 26, '1733202': 20, '1750422': 14, '1662889': 12, '1772839': 4, '1653035': 16, '1713827': 4, '1783165': 6, '1667753': 4, '1713486': 5, '1629601': 23, '1626134': 4, '1699381': 9, '1796802': 4, '1636003': 9, '1683564': 7, '1826712': 24, '1764729': 10, '1818811': 5, '1702450': 5, '1805889': 7, '1713887': 10, '1656849': 17, '1810111': 3, '1793196': 8, '1683470': 10, '1629002': 38, '1730199': 4, '1624336': 6, '1683812': 11, '1695905': 22, '1781433': 20, '1811040': 15, '1769937': 24, '1780917': 9, '1675798': 23, '1786060': 7, '1815404': 3, '1727557': 16, '1629116': 17, '1816538': 5, '1806881': 20, '1666574': 6, '1779229': 5, '1700783': 6, '1820937': 12, '1776465': 16, '1685441': 12, '1689125': 39, '1789165': 19, '1706363': 9, '1683757': 4, '1724117': 4, '1807634': 5, '1697958': 26, '1712345': 4, '1667390': 10, '1748069': 9, '1655264': 9, '1622854': 13, '1710505': 4, '1636725': 11, '1729325': 11, '1697720': 6, '1806164': 3, '1707230': 7, '1795434':